In [46]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt   
from pprint import pprint 

from datetime import datetime, timezone
from dateutil import parser

# Nba 
from nba_api.stats.static import teams, players 
from nba_api.live.nba.endpoints import scoreboard, boxscore, playbyplay

def pp(x, *args):
    pprint(x) if not args else (pprint(x), pp(args))

custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [82]:
# Add endpoints to variable to avoid multiple calls 
board = scoreboard.ScoreBoard()
#box = boxscore.BoxScore('0022100985') 
pbp = playbyplay.PlayByPlay(
    '0022100985') 
    # proxy='https://127.0.0.1:80', 
    # headers=custom_headers, 
    # timeout=100)


In [83]:
game_fx = lambda gid, at, ht, gt: f"{gid}: {at} vs. {ht} @ {gt}" 
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    game_time = parser.parse(game["gameTimeUTC"])
    game_time = game_time.replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(game_fx(game['gameId'], game['awayTeam']['teamName'], 
         game['homeTeam']['teamName'], game_time))
    print(game["gameClock"], game["pbOdds"])
board.games.get_dict()[0].keys()


ScoreBoardDate: 2022-03-09
0022100990: Trail Blazers vs. Jazz @ 2022-03-09 22:00:00-05:00
PT00M00.00S {'team': None, 'odds': 0.0, 'suspended': 1}
0022100985: Lakers vs. Rockets @ 2022-03-09 20:00:00-05:00
PT03M46.00S {'team': None, 'odds': 0.0, 'suspended': 1}
0022100991: Nuggets vs. Kings @ 2022-03-09 22:00:00-05:00
PT11M29.00S {'team': None, 'odds': 0.0, 'suspended': 1}
0022100988: Knicks vs. Mavericks @ 2022-03-09 20:30:00-05:00
PT06M51.00S {'team': None, 'odds': 0.0, 'suspended': 1}
0022100989: Raptors vs. Spurs @ 2022-03-09 20:30:00-05:00
PT03M34.00S {'team': None, 'odds': 0.0, 'suspended': 1}
0022100981: Celtics vs. Hornets @ 2022-03-09 19:00:00-05:00
 {'team': None, 'odds': 0.0, 'suspended': 1}
0022100982: Bulls vs. Pistons @ 2022-03-09 19:00:00-05:00
 {'team': None, 'odds': 0.0, 'suspended': 1}
0022100983: Suns vs. Heat @ 2022-03-09 19:30:00-05:00
 {'team': None, 'odds': 0.0, 'suspended': 1}
0022100984: Hawks vs. Bucks @ 2022-03-09 20:00:00-05:00
 {'team': None, 'odds': 0.0, 's

dict_keys(['gameId', 'gameCode', 'gameStatus', 'gameStatusText', 'period', 'gameClock', 'gameTimeUTC', 'gameEt', 'regulationPeriods', 'ifNecessary', 'seriesGameNumber', 'seriesText', 'homeTeam', 'awayTeam', 'gameLeaders', 'pbOdds'])

In [87]:
#denver = teams.find_teams_by_city("denver")[0]
#print(denver)
#game = box.game.get_dict()
#pp(game.keys())
pbp = playbyplay.PlayByPlay('0022100985') 
line = "{action_number}: {period}: {clock} {player_id} ({action_type})"
# plays are referred to in the live data as `actions`
_actions = pbp.get_dict()['game']['actions'] 
n_actions = len(actions) 
actions = _actions[::-1][:5]  # check ten last actions 
for action in actions:
    player_name = ''
    player = players.find_player_by_id(action['personId'])
    if player is not None:
        player_name = player['full_name']
    print(line.format(
        action_number=action['actionNumber'],
        period=action['period'],
        clock=action['clock'],
        action_type=action['actionType'] + str(action["isFieldGoal"]),
        player_id=player_name))

scores = [(x["clock"], x["scoreHome"], x["scoreAway"]) 
          for x in _actions if x["isFieldGoal"]]
#print(len(scores))
action.keys()

775: 5: PT00M25.70S Malik Monk (3pt1)
773: 5: PT00M29.80S Alperen Sengun (3pt1)
772: 5: PT00M53.20S Austin Reaves (2pt1)
770: 5: PT01M00.00S Jalen Green (3pt1)
769: 5: PT01M23.00S LeBron James (3pt1)


dict_keys(['actionNumber', 'clock', 'timeActual', 'period', 'periodType', 'teamId', 'teamTricode', 'actionType', 'subType', 'qualifiers', 'personId', 'x', 'y', 'side', 'shotDistance', 'possession', 'scoreHome', 'scoreAway', 'edited', 'orderNumber', 'xLegacy', 'yLegacy', 'isFieldGoal', 'shotResult', 'pointsTotal', 'description', 'playerName', 'playerNameI', 'personIdsFilter'])

In [91]:
clk, hpts, apts = zip(*scores)

hpts = np.array(hpts).astype(int)
hpts[::-1]


array([139, 139, 136, 136, 133, 133, 130, 130, 130, 130, 130, 127, 127,
       127, 125, 125, 122, 122, 120, 120, 120, 120, 120, 120, 120, 120,
       120, 118, 116, 116, 116, 114, 114, 114, 114, 114, 114, 111, 109,
       109, 109, 106, 106, 106, 106, 106, 106, 104, 104, 102, 102, 100,
       100, 100, 100, 100, 100, 100,  97,  97,  97,  97,  97,  97,  97,
        95,  93,  93,  93,  91,  91,  91,  91,  91,  91,  91,  89,  88,
        86,  86,  86,  84,  84,  82,  82,  79,  79,  79,  78,  78,  78,
        75,  75,  75,  75,  75,  75,  75,  73,  72,  70,  70,  68,  68,
        68,  68,  65,  65,  65,  65,  63,  63,  61,  61,  61,  61,  61,
        59,  59,  58,  55,  55,  53,  53,  53,  53,  53,  53,  53,  53,
        53,  51,  51,  49,  49,  49,  47,  47,  47,  45,  45,  43,  43,
        40,  40,  40,  40,  38,  35,  35,  35,  32,  32,  32,  30,  30,
        30,  28,  28,  28,  27,  25,  25,  25,  22,  22,  22,  22,  20,
        20,  20,  20,  18,  18,  16,  16,  16,  16,  14,  14,  1